<a href="https://colab.research.google.com/github/AkiChiu/auto_expense_tracking/blob/main/expense_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
from google.colab import auth
from google.auth import default

try:
  import gspread
except ModuleNotFoundError:
  if 'google.colab' in str(get_ipython()):
    %pip install gspread
  import gspread

In [ ]:
from datetime import datetime

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
folder_path = '/content/drive/MyDrive/01_Personal/Finance/'

def change_sign(x):
    return -x

def read_account_a(folder_path):
    file_path = folder_path + '123.csv' #file name has to be updated
    account_a_data = pd.read_csv(file_path)

    # change the -/+ sign in amount column
    account_a_data['Amount'] = account_a_data['Amount'].apply(change_sign)

    # adding and formatting the columns
    account_a_data = account_a_data.assign(
        Wallet="Account A", #update the Account A to the desired bank account
        Item=account_a_data['Description'],
        Category="",
        Currency="GBP",
        Shared="",
        Note=""
    )

    account_a_data['Date'] = pd.to_datetime(account_a_data['Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

    # move 'wallet' to the first order for formatting
    new_order = ['Wallet'] + [col for col in account_a_data.columns if col != 'Wallet']
    account_a_data = account_a_data.reindex(columns=new_order)
    account_a_data = account_a_data[['Wallet', 'Date', 'Item', 'Amount', 'Category', 'Currency', 'Shared', 'Note']]

    return account_a_data

def read_account_b(folder_path):
  file_path = folder_path + '456.csv' #file name has to be updated
  account_b_data = pd.read_csv(file_path)

  #adding and formatting the columns
  account_b_data.columns = ['Date', 'Item', 'Amount']
  account_b_data = account_b_data.assign(
        Wallet="Account B",
        Category="",
        Currency="SGD",
        Shared="",
        Note=""
    )

  #ensuring date format is correct
  account_b_data['Date'] = pd.to_datetime(account_b_data['Date'], format='%d/%m/%Y')
  account_b_data['Date'] = pd.to_datetime(account_b_data['Date']).dt.strftime('%Y-%m-%d')

  #move 'wallet' to the first order for easy pasting
  new_order = ['Wallet'] + [col for col in account_b_data.columns if col != 'Wallet']
  account_b_data = account_b_data.reindex(columns=new_order)
  return account_b_data

def process_data(data, wallet_name, column_mapping):
    data = data.rename(columns=column_mapping)
    data['Wallet'] = wallet_name
    data['Category'] = ""
    data['Currency'] = "TWD"
    data['Shared'] = ""
    data['Note'] = ""

    # remove unnecessary strings from the data and convert to integer
    data['Amount'] = data['Amount'].apply(lambda x: int(re.sub(r'\D', '', str(x))))

    # filter for required columns
    data = data[['Wallet', 'Date', 'Item', 'Amount', 'Category', 'Currency', 'Shared', 'Note']]

    # ensure the date column is in the correct format
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
    data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')

    # filter data for the current year and month
    current_date = datetime.now()
    current_month = current_date.month
    current_year = current_date.year

    data['Date'] = pd.to_datetime(data['Date'])
    condition = (data['Date'].dt.month == current_month) & (data['Date'].dt.year == current_year)
    return data[condition]

def read_account_c_debit(folder_path):
    file_path = folder_path + '789.xls' #file name has to be updated
    account_c_db_data = pd.read_excel(file_path, sheet_name='帳戶明細')
    column_mapping = {'帳務日期': 'Date', '備註': 'Item', '金額': 'Amount'}  # 欄位名稱的映射表
    return process_data(account_c_db_data, "Account C Debit", column_mapping)

def read_account_c_credit(folder_path):
    file_path = folder_path + '789.xls' #file name has to be updated
    account_c_cr_data = pd.read_excel(file_path, sheet_name='信用卡明細')
    column_mapping = {'入帳日期': 'Date', '消費明細': 'Item', '金額': 'Amount'}  # 欄位名稱的映射表
    return process_data(account_c_cr_data, "Account C Credit", column_mapping)

In [ ]:
df1 = read_account_a(folder_path)
df2 = read_account_b(folder_path)
df3 = read_account_c_debit(folder_path)
df4 = read_account_c_credit(folder_path)


df1 = df1.sort_values('Date', ascending=True)
df2 = df2.sort_values('Date', ascending=True)
df3 = df3.sort_values('Date', ascending=True)
df4 = df4.sort_values('Date', ascending=True)

concatenated_df = pd.concat([df1, df2, df3, df4])
concatenated_df['Date'] = pd.to_datetime(concatenated_df['Date']).dt.strftime('%Y-%m-%d')


concatenated_df

In [ ]:
worksheet = gc.open("[Copy] Example Expense Template")
sheet = worksheet.worksheet("Expenses")

# read the existing data in the sheet
existing_data = sheet.get_all_values()

# find the last filled row
last_row_index = len(existing_data) + 1

# update the format of concatenated_df
data_to_insert = concatenated_df.values.tolist()

# add the data to the next empty row
sheet.append_rows(data_to_insert, value_input_option='RAW', insert_data_option='INSERT_ROWS', table_range=f'B{last_row_index}')